In [1]:
!pip install matplotlib numpy pandas seaborn scikit-learn ipython
!pip install --upgrade kneed

import numpy as np
import math as mt
import pandas as pd
import random
import matplotlib.pyplot as plt
from queue import PriorityQueue
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

avgroundrobin=0
avgdelaygsa=0
avgdelayrandom=0

for itr in range(100):
  #Initiation
  class SDN:
      def __init__(self, index, x, y, CPUcyc):
          self.id = index
          self.x = x
          self.y = y
          self.tasks_incoming = []
          self.tasks_priority_queue = PriorityQueue()
          self.fcfs_queue = PriorityQueue()
          self.sjf_queue = PriorityQueue()
          self.random_queue = PriorityQueue()
          self.list_of_fns = []
          self.list_of_eus = []
          self.CPUcyc = CPUcyc

  class FN:
      def __init__(self, index, x, y, freq, CPUcyc, failrate):
          self.id = index
          self.x = x
          self.y = y
          self.sdn_num = None
          self.freq = freq
          self.CPUcyc = CPUcyc
          self.failrate = failrate

  class EU:
      def __init__(self, index, x, y):
          self.id = index
          self.x = x
          self.y = y
          self.sdn_num = None
          self.tasks = []

  T = []
  N = 3000
  M = 250
  eus = [] # list of all end users
  fns = [] # list of all fog nodes
  sdns = [] # list of all sdns
  tasks = []
  eus_pos = []
  fns_pos = []
  Co={}
  SCo={}
  Clu={}

  # calculating failure rate of fog nodes using poisson distribution
  def computeFailRate():
      _lambda = 25
      p = random.random()
      failrate = -(mt.log(1.0-p)/_lambda)
      return failrate
  for i in range(M):
      x = random.random()*1000000
      y = random.random()*1000000
      freq = random.randint(1, 15)
      CPUcyc = random.randint(500, 1000)
      fns.append(FN(i, x, y, freq, CPUcyc, computeFailRate()))
      fns_pos.append([x,y])
  for i in range(N):
      x = random.random()*1000000
      y = random.random()*1000000
      eus.append(EU(i, x, y))
      eus_pos.append([x,y])
  # plt.figure(facecolor='white')
  # ax=plt.axes()
  # ax.set_facecolor("white")
  # plt.rcParams["figure.figsize"] = [9.00, 9.00]
  # plt.rcParams["figure.autolayout"] = True

  fn_np = np.array(fns_pos)
  eu_np = np.array(eus_pos)
  x, y = fn_np.T
  # plt.plot(x, y, 'go', ms=7)
  x, y = eu_np.T
  # plt.plot(x, y, 'bo', ms=4)

  # plt.title('Unclustered Fog Nodes')
  # plt.show()



  # Initially finding optimum number of clusters/SDNs - ELBOW METHOD
  sse = []
  max_clusters = int(M/4)
  for k in range(1, max_clusters):
      kmeans = KMeans(n_clusters=k, init= 'k-means++', n_init=10, max_iter=300, random_state=42)
      kmeans.fit(fn_np)
      sse.append(kmeans.inertia_)
  # plt.style.use("fivethirtyeight")
  # plt.plot(range(1, max_clusters), sse)
  # plt.xticks(range(1, max_clusters))
  # plt.xlabel("Number of Clusters")
  # plt.ylabel("SSE")
  # plt.show()
  kl = KneeLocator(
      range(1, max_clusters), sse, curve="convex", direction="decreasing"
  )
  # print(kl.elbow)



  #Fog Nodes clustered (K Means)
  kmeans = KMeans(
      init="random",
      n_clusters=kl.elbow,
      n_init=10,
      max_iter=300,
      random_state=42
    )
  y_predict = kmeans.fit_predict(fn_np)
  # plt.figure(facecolor='white')
  # ax=plt.axes()
  # ax.set_facecolor("white")
  for i in range(0,kl.elbow):
    color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])]
    # plt.scatter(fn_np[y_predict == i, 0], fn_np[y_predict == i, 1], s = 100, c = color)

  # plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 300, c = 'yellow', label = 'SDNs')
  # plt.title('Clusters of FNs')

  # plt.legend()
  # plt.show()

  for ind, centers in enumerate(kmeans.cluster_centers_):
      CPUcyc = random.randint(1000, 2000)
      sdns.append(SDN(ind, centers[0], centers[1], CPUcyc))

  for fn_num, sdn_num in enumerate(y_predict):
    sdns[sdn_num].list_of_fns.append(fns[fn_num])
    fns[fn_num].sdn_num = sdn_num



  # EUs clustered according to governing SDNs (according to closest distance)
  for i in range(N):
      Co[i]=[]
      for j in range(len(sdns)):
          d=mt.sqrt((eus[i].x-sdns[j].x)**2+(eus[i].y-sdns[j].y)**2)
          Co[i].append((j,d))
      Co[i]=sorted(Co[i], key=lambda x: x[1])

  for i in range(N):
    num = Co[i][0][0]
    eus[i].sdn_num = num
    sdns[num].list_of_eus.append(eus[i])

  x, y = eu_np.T
  # for i in range(N):
  #   if(eus[i].sdn_num==0):
  #     plt.plot(x[i],y[i],'bo',ms=3)
  #   elif(eus[i].sdn_num==1):
  #     plt.plot(x[i],y[i],'ro',ms=3)
  #   elif(eus[i].sdn_num==2):
  #     plt.plot(x[i],y[i],'go',ms=3)
  #   elif(eus[i].sdn_num==3):
  #     plt.plot(x[i],y[i],'co',ms=3)
  #   elif(eus[i].sdn_num==4):
  #     plt.plot(x[i],y[i],'mo',ms=3)

  # plt.title('EUs under different SDNs')
  # plt.show()

  # for eu in eus:
  #   print(str(eu.x) + " " + str(eu.y) +  " " + str(eu.sdn_num))


  #Generate tasks
  import sched, time
  import pandas as pd
  from time import mktime

  class taskers:
      def __init__(self, id, gen_eu, gen_sdn, gen_time, deadline, size, freq, CPUcap, num):
          self.id = id
          self.gen_eu = gen_eu
          self.gen_sdn = gen_sdn
          self.gen_time = gen_time
          self.deadline = deadline
          self.size = size
          self.freq = freq
          self.CPUcap = CPUcap
          self.queue_pos = 0

      def __lt__(self, obj):
          """self < obj."""
          return self.id < obj.id



  _lambda = 5
  arrival_time = time.time()
  cnt = 0

  tasks.clear()

  for sdn in sdns:
    sdn.tasks_incoming.clear()

  for eu in eus:
    eu.tasks.clear()

  def make_task(gen_eu, gen_sdn, count, arrival_time):
    random_time = arrival_time + random.uniform(250, 500)
    deadline = random_time
    size = random.randint(500, 2000) # in Bytes
    freq = 2.5 * pow(10, 2) # in MHz
    CPUcap = random.randint(500, 1000)
    return taskers(count, gen_eu, gen_sdn, arrival_time, deadline, size, freq, CPUcap, 0)

  for sdn in sdns:
    lst = []
    for eu in sdn.list_of_eus:
      num = random.randint(0, 3) # number of tasks generated at each EU can vary from 0 to 3
      for i in range(num):
        p = random.random()
        inter_arrival_time = -(mt.log(1.0-p)/_lambda)
        # arrival_time = time.mktime(arrival_time.timetuple()) + arrival_time.microsecond/1e6 + inter_arrival_time
        arrival_time = arrival_time + inter_arrival_time
        task = make_task(eu.id, sdn.id, cnt, arrival_time)
        eu.tasks.append(task)
        sdn.tasks_incoming.append(task)
        tasks.append(task)
        cnt = cnt+1

  for sdn in sdns:
    # print(str(len(sdn.tasks_incoming))+" Tasks incoming at SDN "+str(sdn.id)+" :")
    sdn.tasks_priority_queue = PriorityQueue()
    for task in sdn.tasks_incoming:
      priority = 1/((task.deadline - task.gen_time) - (task.size/task.CPUcap))
      k = (priority, task)
      sdn.tasks_priority_queue.put(k)

  for sdn in sdns:
    sdn.fcfs_queue = PriorityQueue()
    for task in sdn.tasks_incoming:
      priority = 1/task.gen_time
      k = (priority, task)
      sdn.fcfs_queue.put(k)

  for sdn in sdns:
    sdn.sjf_queue = PriorityQueue()
    for task in sdn.tasks_incoming:
      priority = 1/(task.size/task.CPUcap)
      k = (priority, task)
      sdn.sjf_queue.put(k)

  for sdn in sdns:
    sdn.random_queue = PriorityQueue()
    for task in sdn.tasks_incoming:
      priority = random.random()
      k = (priority, task)
      sdn.random_queue.put(k)

  # print("Count of tasks is " + str(cnt))


  def computeQueueingDelay(task, sdn):
    return task.freq * task.queue_pos / sdn.CPUcyc


  def computeOffloadingDelay(task, fn):
    return task.size * task.freq / fn.CPUcyc


  import math
  class Channel:
    W = math.pow(10,7)
    No = 12.59 * math.pow(10,-15)
    h = 0.50
    p0 = 0.758
    gamma = 0.01
    d0 = 1000
    interPower = 50.119 * math.pow(10,-9)  # -109dBm
    shadowFactor = 0.3162
    commFailure = 0.002

    def get_pathloss(self, x1, y1, x2, y2):
      d = math.sqrt(math.pow(x2 - x1, 2)+ math.pow(y2 - y1,2))
      return self.p0 + (10 * self.gamma * math.log10(d/self.d0))

    def get_bw(self,power,x1,y1,x2,y2):
      pathLoss = self.get_pathloss(x1,y1,x2,y2)
      # d = math.sqrt(math.pow(x2 - x1, 2) + math.pow(y2 - y1, 2))
      #bw = math.log2(1+((self.h * power * math.pow(1/d,pathLoss))/(self.No)))
      bw = math.log2(1+((self.shadowFactor * power * pathLoss)/(self.interPower + (self.W* self.No))))
      return bw

  def computeTransmissionDelay(task, fn):
    gen_eu = eus[task.gen_eu]
    ch= Channel()
    return ch.get_bw(0.5, gen_eu.x, gen_eu.y, fn.x, fn.y)



  sdn = sdns[0]
  num_tasks = 300 # the number of tasks we are taking from the priority queue at any instance and assigning it

  connections = {}
  for i in range(num_tasks):
      task = sdn.tasks_priority_queue.get()[1]
      allowed_delay = task.deadline-task.gen_time
      task.queue_pos = i+1
      connections[i] = {}
      connections[i]['task_id'] = task.id
      connections[i]['fns'] = []
      qd = computeQueueingDelay(task, sdn)
      for fn in sdn.list_of_fns:
          if(fn.failrate > 0.05):
            pass
          od = computeOffloadingDelay(task, fn)
          td = computeTransmissionDelay(task, fn)
          total_delay = qd + od + td
          # print("qd " + str(qd))
          # print("od " + str(od))
          # print("td " + str(td))
          # print("total " + str(total_delay))
          if(total_delay <= allowed_delay):
            connections[i]['fns'].append(fn.id)

  # for item in connections.items():
  #   print(item)



  from matplotlib.markers import JoinStyle
  from pprint import pprint
  from matplotlib import pyplot as plt

  import numpy as np


  class GSA:
      def __init__(self, pop_size=1, tasks=1, max_iter=100):
          self.cost_func = None

          self.alpha = 0.1
          self.G = 0.9
          self.max_iter = max_iter
          self.pop_size = pop_size
          self.tasks = tasks
          self.best_so_far = None

          self.X = np.random.rand(pop_size, tasks)
          self.V = np.random.rand(pop_size, tasks)
          self.f = np.full((pop_size, tasks), None)  # None  # Will become a list inside cal_f
          self.a = np.full((pop_size, tasks), None)
          self.q = np.full((pop_size, 1), None)
          self.M = np.full((pop_size, 1), None)
          self.cost_matrix = np.full((pop_size, 1), None)
          self.max_delay = None
          self.avg_delay = None

      # generates initial population
      def generate_initial_pop(self):
          for i in range(self.pop_size):
              for j in range(0, self.tasks):
                  self.X[i][j] = random.uniform(0, len(connections[j]['fns'])-1)

      # Evaluate a single x (x_i)
      def evaluate(self, args):
          return self.cost_func(args)

      # Generate the cost of all particles
      def gen_cost_matrix(self):
          for i, x in enumerate(self.X):
            self.cost_matrix[i] = self.evaluate(x)

      def cal_q(self):
          best = np.min(self.cost_matrix)
          worst = np.max(self.cost_matrix)

          self.q = (self.cost_matrix - worst) / best - worst

      def cal_m(self):
          self.M = self.q / np.sum(self.q)

      def cal_f(self):
          costs = self.cost_matrix.copy()
          costs.sort(axis=0)
          costs = costs

          for i in range(self.pop_size):
              f = None
              for cost in costs:
                  tmp = np.where(self.cost_matrix == cost)[0]
                  j = tmp[0]

                  dividend = float(self.M[i] * self.M[j])
                  divisor = np.sqrt(np.sum((self.X[i] - self.X[j]) ** 2)) + np.finfo('float').eps
                  if f is None:
                      f = self.G * (dividend / divisor) * (self.X[j] - self.X[i])
                  else:
                      f = f + self.G * (dividend / divisor) * (self.X[j] - self.X[i])

              self.f[i] = np.random.uniform(0, 1) * f

      def cal_a(self):
          for i in range(self.pop_size):
              self.a[i] = self.f[i] / self.M[i]

      def cal_v(self):
          self.V = (np.random.uniform(0, 1) * self.V) + self.a

      def move(self):
          self.X = self.X + self.V

      def update_g(self, iteration):
          self.G = self.G * np.e ** (- self.alpha * (iteration / self.max_iter))

      def show_results(self):
          global avgdelaygsa
          self.avg_delay = self.evaluate(self.best_so_far)/num_tasks
          avgdelaygsa+=self.avg_delay
          # self.max_delay = find_max_cost(self.best_so_far)
          # print('Best seen so far is located at: ', self.best_so_far, 'Cost:', self.evaluate(self.best_so_far))
          # print('Average delay:', self.avg_delay)
          # print('Max delay: ', self.max_delay)


      def update_best_so_far(self):
          best = np.min(self.cost_matrix)
          tmp = np.where(self.cost_matrix == best)[0]
          index = tmp[0]
          # if self.best_so_far is not None:
          #     print("best " + str(self.evaluate(self.best_so_far)))

          if self.best_so_far is None or self.evaluate(self.best_so_far) > best:
              self.best_so_far = self.X[index]

      def start(self):
          _iter = 0
          self.generate_initial_pop()
          avg = []
          bsf = []
          while _iter < self.max_iter:
              self.gen_cost_matrix()
              self.cal_q()
              self.cal_m()
              self.update_g(_iter)
              self.cal_f()
              self.cal_a()
              self.cal_v()
              self.move()
              self.update_best_so_far()

              avg.append(sum(self.cost_matrix) / len(self.cost_matrix))
              bsf.append(self.evaluate(self.best_so_far))

              _iter += 1
          else:
              self.show_results()
              # print("The optimal assignment is as follows: ")
              optim = self.best_so_far
              # for task_idx, fn_idx in enumerate(optim):
              #     if len(connections[task_idx]['fns']) != 0:
              #         print("Task " + str(connections[task_idx]['task_id']) + " is assigned to fog node " + str(connections[task_idx]['fns'][int(fn_idx)]))
              #     else:
              #         print("Task " + str(connections[task_idx]['task_id']) + " is not assigned")
              # plt.figure()
              # avg_plt = plt.subplot(1, 2, 1)
              # avg_plt.plot(avg, color='gold')
              # bsf_plt = plt.subplot(1, 2, 2)
              # bsf_plt.plot(bsf, color='darkviolet')
              # plt.show()


  # Minimize function
  def fitness_func(args):
      pop = args
      cost = 0
      for task_idx, fn_idx in enumerate(pop):
          if(len(connections[task_idx]['fns'])) != 0:
              task = tasks[connections[task_idx]['task_id']]
              fn_idx = min(int(fn_idx), len(connections[task_idx]['fns'])-1 )
              fn = fns[connections[task_idx]['fns'][int(fn_idx)]];
              delay = computeOffloadingDelay(task, fn) + computeQueueingDelay(task, sdns[0]) + computeTransmissionDelay(task, fn)
              cost += 0.5*delay + 0.5*fn.failrate
      return cost

  def find_max_cost(args):
      pop = args
      mx = 0
      for task_idx, fn_idx in enumerate(pop):
          if(len(connections[task_idx]['fns'])) != 0:
              task = tasks[connections[task_idx]['task_id']]
              fn_idx = min(int(fn_idx), len(connections[task_idx]['fns'])-1 )
              fn = fns[connections[task_idx]['fns'][int(fn_idx)]];
              delay = computeOffloadingDelay(task, fn) + computeQueueingDelay(task, sdns[0]) + computeTransmissionDelay(task, fn)
              mx = max(mx, delay)
      return mx

  gsa = GSA(pop_size=20, tasks=num_tasks, max_iter=300)
  gsa.cost_func = fitness_func
  gsa.start()

  tot_random_delay=0
  for i in range(num_tasks):
    if(len(connections[i]['fns'])) != 0:
      sz=len(connections[i]['fns'])
      fn_idx=random.randint(0,sz-1)
      fn = fns[connections[i]['fns'][int(fn_idx)]]
      delay = computeOffloadingDelay(task, fn) + computeQueueingDelay(task, sdns[0]) + computeTransmissionDelay(task, fn)
      tot_random_delay += 0.5*delay + 0.5*fn.failrate
  avgdelayrandom+=tot_random_delay/num_tasks


  #Round-Robin Assignment (for average Delay calculation)
  tot_roundrobin_delay=0
  sdn=sdns[0]
  map={}
  #initialise the map
  for fn in sdn.list_of_fns:
    map[fn.id]=1
  def update_map():
    for fn in sdn.list_of_fns:
      map[fn.id]+=1
  for i in range(num_tasks):
    if(len(connections[i]['fns'])) != 0:
      fn_id=connections[i]['fns'][0]
      for j in connections[i]['fns']:
            if map[fn_id]<map[j]:
              fn_id=j
      delay = computeOffloadingDelay(task, fn) + computeQueueingDelay(task, sdns[0]) + computeTransmissionDelay(task, fn)
      tot_roundrobin_delay += 0.5*delay + 0.5*fn.failrate
      map[fn_id]=0
      update_map()
  avgroundrobin+=tot_roundrobin_delay/num_tasks
  print(itr+1)


print(avgdelaygsa/100)
print(avgdelayrandom/100)
print(avgroundrobin/100)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
113.22415495628377
149.222693176436
163.53375746726752
